# Workflow notebook for the PolymerXtal SimTool #
### Tongtong Shen, Jessica Nash and Alejandro Strachan ###
### School of Materials Engineering, Purdue University and The Molecular Sciences Software Institute###

The PolymerXtal SimTool is based on PolymerXtal software designed to build and analyze molecular-level polymer crystal structures. PolymerXtal provides a standardized process to generate polymer crystal structure based on monomer, tacticity, helicity, chiriality and unit cell information and analyze the crystallinity in polymer systems with given atom trajectories. These features have allowed PolymerXtal to lead further investigations of semi-crystalline polymers where most properties are governed by the crystalline microstructure and promote future research endeavors in the area of crystalline polymer simulations.

In general, users need to specify as inputs the polymer type they wish to run, the helice structure, tacticity and chiriality. The SimTool generates the chain helix structure and a visual snapshot of it.

The goal of the workflow notebook is to set up and run the PolymerXtal SimTool, which is demonstrated both for the case a single build, or as a high-throughput study, exploring various polymer types and helicity at once.

## Set up imports ##

We will first import the necessary libraries and packages, including the simtool package which helps locate the SimTool we wish to run and pass it inputs and outputs

In [1]:
from hublib.tool import get_inputs, run_simtool, rdisplay, read, get_outputs_df, get_outputs, get_output_files
from simtool import findInstalledSimToolNotebooks, searchForSimTool, findSimToolNotebook
from simtool import getSimToolInputs, getSimToolOutputs, Run

import os

## SimTool information ##

We will first create a dictionary containing the SimTool information using the "findSimToolNotebook" function

In [2]:
# Identify the simtool and retrieve its status
simToolName = "polymerxtal"
nb = searchForSimTool('polymerxtal')
for key in nb.keys():
    print("%18s = %s" % (key,nb[key]))

      notebookPath = /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       simToolName = polymerxtal
   simToolRevision = None
         published = False


### SimTool Inputs and Outputs ###
We will then create an object for the simtool inputs using the "getSimToolInputs" function - this object is validated by the simtool library so only allowed values are passed to the simtool

In [3]:
# get the list of inputs
inputs = getSimToolInputs(nb)
print(inputs)

polymer_type:
    type: Text
    description: Select the polymer type.
    options: ['PAN', 'PE', 'PP', 'PS', 'POM', 'PTFE', 'PVC']
    value: PAN

helicity:
    type: Text
    description: Select the helicity of the polymer chain.
    options: ['Helix 2*3/1', 'Helix 2*2/1', 'Helix 2*1/1']
    value: Helix 2*3/1

monomers:
    type: Number
    description: Number of monomers
    min: 1
    max: 100
    value: 30

tacticity:
    type: Text
    description: Select the tacticity of the polymer chain.
    options: ['isotactic', 'atactic', 'syndiotactic', 'None']
    value: isotactic

chiriality:
    type: Text
    description: Select the chiriality of the polymer chain.
    options: ['left', 'right', 'None']
    value: right

head_tail_defect_ratio:
    type: Number
    description: Ratio of head-to-head and tail-to-tail connections
    min: 0
    max: 1
    value: 0

configs:
    type: Number
    description: Number of attempts to find a configuration
    min: 1
    max: 5000
    value: 3

In [4]:
# we can get a list of expected outputs
getSimToolOutputs(nb)

HelixName:
    type: Text

PDBview:
    type: Text

PDB:
    type: Text

LAMMPSDataFile:
    type: Text

Datafile_warnings:
    type: Text

X6paircoeffs:
    type: Text

LAMMPSinputfile:
    type: Text

## Setting SimTool Inputs ##

Now we will setup the inputs and run a first simulation. Important aspects on SimTool inputs are:
 * Inputs are validated

In [5]:
# range checking
inputs.monomers.value = 6000

ValueError: Maximum value is 100

## Run the SimTool ###

In [6]:
inputs.polymer_type.value = 'PE'
inputs.infinite.value = False
inputs.create_lmpdata_file.value = False
inputs.create_lmpinput_file.value = False

In [7]:
r = Run(nb, inputs)

runname   = b165d92490274fae99554be9e4ad1859
outdir    = RUNS/b165d92490274fae99554be9e4ad1859
cached    = False
published = False
submit --local papermill -f inputs.yaml
       /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       polymerxtal.ipynb


Input Notebook:  /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
Output Notebook: polymerxtal.ipynb
Executing: 100%|██████████| 13/13 [00:38<00:00,  2.94s/cell]


The following outputs are missing: ['LAMMPSinputfile', 'LAMMPSDataFile', 'Datafile_warnings', 'X6paircoeffs']


## Explore PolymerXtal outputs ##
The Run object contains a useful method called "getResultSummary", which we can use to view the SimTool outputs at a high level in a pandas dataframe.

In [8]:
r.getResultSummary()

,name,data,encoder,display,filename
0,simToolSaveErrorOccurred,0,text,None,polymerxtal.ipynb
1,simToolAllOutputsSaved,0,text,None,polymerxtal.ipynb
2,HelixName,"""PE_helix_2_3_1_iso+""",text,None,polymerxtal.ipynb
3,PDBview,file://PE_helix_2_3_1_iso+_view.pdb,text,None,polymerxtal.ipynb
4,PDB,file://PE_helix_2_3_1_iso+.pdb,text,None,polymerxtal.ipynb


In [9]:
# Let's look at the final structure first
pdbFile = r.read('PDBview',raw=True)
import imolecule
imolecule.draw(pdbFile[7:],size=(1000, 300))
#imolecule.draw(r.outdir+"/"+r.read('Helix Name')+".pdb",size=(1000, 300))#, camera_type="orthographic") 

## Scripting multiple PolymerXtal builds ##

The above cells demonstrated a single build using the simtool library. We will now demonstrate a high-throughput approach to setting up multiple builds, where will demonstrated various polymer types and helices.

In [10]:
polymer_type_list = ['PP', 'PS']
inputs.infinite.value = True
inputs.create_lmpdata_file.value = True
inputs.create_lmpinput_file.value = True

for i,polymer_type in enumerate(polymer_type_list):
    print ("Polymer Type: ", polymer_type)
   
    inputs.polymer_type.value = polymer_type
    if i:
        inputs.ffield.value = "PCFF"
    
    # run the SimTool with the input values
    r = Run(nb, inputs)
    

Polymer Type:  PP
runname   = 76e6a68219a9475888321eab0659da2c
outdir    = RUNS/76e6a68219a9475888321eab0659da2c
cached    = False
published = False
submit --local papermill -f inputs.yaml
       /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       polymerxtal.ipynb


Input Notebook:  /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
Output Notebook: polymerxtal.ipynb
Executing:  62%|██████▏   | 8/13 [00:12<00:08,  1.62s/cell]1 molecule converted
1 molecule converted
Executing: 100%|██████████| 13/13 [00:57<00:00,  4.41s/cell]


The following outputs are missing: ['Datafile_warnings']
Polymer Type:  PS
runname   = f21a3648e4a646d6bc6e0237b2b27e58
outdir    = RUNS/f21a3648e4a646d6bc6e0237b2b27e58
cached    = False
published = False
submit --local papermill -f inputs.yaml
       /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       polymerxtal.ipynb


Input Notebook:  /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
Output Notebook: polymerxtal.ipynb
Executing:  62%|██████▏   | 8/13 [00:14<00:09,  1.82s/cell]1 molecule converted
1 molecule converted
Executing: 100%|██████████| 13/13 [03:40<00:00, 16.99s/cell]


The following outputs are missing: ['Datafile_warnings', 'X6paircoeffs']


In [11]:
#visualize final snapshot of structure
pdbFile = r.read('PDBview',raw=True)
imolecule.draw(pdbFile[7:],size=(1000, 300))
#imolecule.draw(r.outdir+"/"+r.read('Helix Name')+".pdb",size=(1000, 300))#, camera_type="orthographic") 